In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization


keras.utils.set_random_seed(111)

2024-07-14 22:48:14.516404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 22:48:14.534824: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 22:48:14.534861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-14 22:48:14.546411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-14 22:48:15.205487: W tensorflow/compiler/tf

In [2]:
# Test keras version and cuda installation
for each in tf.config.list_physical_devices():
    print(each)
    
print("Current tensorflow ->", tf.__version__)
print("Current keras ->", keras.__version__)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
Current tensorflow -> 2.16.2
Current keras -> 3.4.1


2024-07-14 22:48:17.290020: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-14 22:48:17.290262: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-14 22:48:17.323443: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

```shell
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip
!unzip -qq Flickr8k_Dataset.zip
!unzip -qq Flickr8k_text.zip
!rm Flickr8k_Dataset.zip Flickr8k_text.zip
```

In [3]:
def get_image_path(workstation: str) -> str:
    if workstation == "office":
        IMAGE_PATH = "/media/loveplay1983/data/ML/imgcap/Flickr/flickr8k/Flicker8k_Dataset"
    else: 
        IMAGE_PATH = "/media/dllab/AI-PhD-Study/test-dataset/imgcap/Flickr/flickr8k_/Flickr8k_Dataset"
    return IMAGE_PATH

def get_cap_path(workstation: str) -> str:
    if workstation == "office":
        CAP_PATH = "/media/loveplay1983/data/ML/imgcap/Flickr/flickr8k/Flickr8k_text"
    else:
        CAP_PATH = "/media/dllab/AI-PhD-Study/test-dataset/imgcap/Flickr/flickr8k_/Flickr8k_text"
    return CAP_PATH

IMAGE_SIZE = (229, 229)
VOCAB_SIZE = 10000 # text vocab size
SEQ_LENGTH = 25    # fixed length for the text sequence
EMBED_DIM = 512 # d_model, feature embedding dimension
FF_DIM = 512 # per layer units in the feed forward 
BATCH_SIZE = 64
EPOCHS = 30
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
# dataset preparation

def load_captions_data(filename):
    """
    Loads captions(text) data and maps them to the corresponding images
    Args:
        filename: Path to the text file containing caption data
    Returns: 
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """

    with open(filename) as caption_file:
        caption_data = caption_file.readlines()
        caption_mapping={}
        text_data=[]
        images_to_skip = set()

        for line in caption_data:
            line = line.rstrip("\n")
            img_name, caption = line.split("\t")  # img name  - captions
    
            img_name = img_name.split("#")[0]
            img_name = os.path.join(get_image_path("home"), img_name.strip())
    
            tokens = caption.strip().split()
    
            if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
                """
                skip image data which are linked to too short or too long caption
                """
                images_to_skip.add(img_name)
                continue
    
            if img_name.endswith("jpg") and img_name not in images_to_skip:
                caption = "<start> " + caption.strip() + " <end>"
                text_data.append(caption)
    
                if img_name in caption_mapping:
                    caption_mapping[img_name].append(caption)
                else:
                    caption_mapping[img_name] = [caption]

        for img_name in images_to_skip:
            if img_name in caption_mapping:
                del caption_mapping[img_name]

        return caption_mapping, text_data

In [5]:
def train_val_split(caption_data, train_size=.8, shuffle=True):
    """
    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # Get the list of all images
    all_images = list(caption_data.keys())

    # shuffle
    if shuffle:
        np.random.shuffle(all_images)

    # data split
    train_size = int(len(caption_data) * train_size)

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }

    validation_data  = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:]
    }

    # return the splits
    return training_data, validation_data

In [6]:
# load dataset
cap_data = os.path.join(get_cap_path("home"), "Flickr8k.token.txt")
print(cap_data)

captions_mapping, text_data = load_captions_data(cap_data)

train_data, valid_data = train_val_split(captions_mapping)
print(f"Number of training samples: {len(train_data)}")
print(f"Number of validation samples: {len(valid_data)}")

/media/dllab/AI-PhD-Study/test-dataset/imgcap/Flickr/flickr8k_/Flickr8k_text/Flickr8k.token.txt
Number of training samples: 6114
Number of validation samples: 1529


In [7]:
type(train_data)

dict

In [8]:
# text_data

In [9]:
# Vectorizing the text data

strip_chars = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
strip_chars = strip_chars.replace("<", "")
strip_chars = strip_chars.replace(">", "")


In [10]:

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

vectorization = TextVectorization(
    max_tokens = VOCAB_SIZE,
    output_mode = "int",
    output_sequence_length = SEQ_LENGTH,
    standardize = custom_standardization
)

vectorization.adapt(text_data)

# Data augmentation
image_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(.2),
        layers.RandomContrast(.3),
    ]
)


2024-07-14 22:48:22.445011: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-14 22:48:22.445213: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-14 22:48:22.445363: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [11]:
# Build tf.data.Dataset pipeline for training
# 1. read the image form the disk
# 2. tokenize the five captions corresponding to the image

def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def process_input(img_path, captions):
    return decode_and_resize(img_path), vectorization(captions)

def make_dataset(images, captions):
    dataset = tf.data.Dataset.from_tensor_slices((images, captions))
    dataset = dataset.shuffle(BATCH_SIZE * 8)
    dataset = dataset.map(process_input, num_parallel_calls = AUTOTUNE)
    return dataset

In [12]:
train_dataset = make_dataset(list(train_data.keys()), list(train_data.values()))
valid_dataset = make_dataset(list(valid_data.keys()), list(valid_data.values()))

In [13]:
len(train_dataset), len(valid_dataset)

(6114, 1529)

In [14]:
type(train_dataset)

tensorflow.python.data.ops.map_op._ParallelMapDataset

In [15]:
print(train_dataset)

<_ParallelMapDataset element_spec=(TensorSpec(shape=(229, 229, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>


In [43]:
dataset = train_dataset.batch(32)
for batch in dataset:
    print(batch[0].shape)

(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 229, 3)
(32, 229, 

2024-07-14 23:01:30.318754: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
# # Build the model

# def get_cnn_model():
#     base_model = efficientnet.EfficientNetB0(
#         input_shape=(*IMAGE_SIZE, 3),
#         include_top=False,
#         weights="imagenet"
#     )
#     # We freeze our feature extractor
#     base_model.trainable = False
#     base_model_out = base_model.output
#     base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
#     cnn_model = keras.models.Model(base_model.input, base_model_out)
#     return cnn_model

In [17]:
# class TransformerEncoderBlock(layers.Layer):
#     def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
#         super().__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.dense_dim = dense_dim
#         self.num_heads = num_heads
#         self.attention_1 = layers.MultiHeadAttention(
#             num_heads = num_heads, 
#             key_dim=embed_dim,
#             dropout=0.0
#         )
#         self.layernorm_1 = layers.LayerNormalization()
#         self.layernorm_2 = layers.LayerNormalization()
#         self.dense_1 = layers.Dense(embed_dim, activation="relu")

#     def call(self, inputs, training, mask=None):
#         inputs = self.layernorm_1(inputs)
#         inputs = self.dense_1(inputs)
#         attention_output_1 = self.attention_1(
#             query=inputs,
#             value=inputs,
#             key=inputs,
#             attention_mask=None,
#             training=training
#         )
#         out_1 = self.layernorm_2(inputs + attention_output_1)
#         return out_1

In [18]:
# class PositionalEmbedding(layers.Layer):
#     def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
#         super().__init__(**kwargs)
#         self.token_embeddings = layers.Embedding(
#             input_dim=vocab_size,
#             output_dim = embed_dim,
#         )
#         self.position_embeddings = layers.Embedding(
#             input_dim = sequence_length, 
#             output_dim = embed_dim
#         )
#         self.sequence_length = sequence_length
#         self.vocab_size = vocab_size
#         self.embed_dim = embed_dim
#         self.embed_scale = tf.math.sqrt(tf.cast(embed_dim, tf.float32))

#     def call(self, inputs):
#         length = tf.shape(inputs)[-1]
#         positions = tf.range(start=0, limit=length, delta=1)
#         embedded_tokens = self.token_embeddings(inputs)
#         embedded_tokens = embedded_tokens * self.embed_scale
#         embedded_positions = self.position_embeddings(positions)
#         return embedded_tokens + embedded_positions

#     def compute_mask(self, inputs, mask=None):
#         return tf.math.not_equal(inputs, 0)


In [19]:
# class TransformerDecoderBlock(layers.Layer):
#     def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
#         super().__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.ff_dim = ff_dim
#         self.num_heads = num_heads
#         self.attention_1 = layers.MultiHeadAttention(
#             num_heads = num_heads, 
#             key_dim=embed_dim,
#             dropout=.1,
#         )
#         self.attention_2 = layers.MultiHeadAttention(
#             num_heads = num_heads,
#             key_dim=embed_dim,
#             dropout =.1,
#         )
#         self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
#         self.ffn_layer_2 = layers.Dense(embed_dim)
        
#         self.layernorm_1 = layers.LayerNormalization()
#         self.layernorm_2 = layers.LayerNormalization()
#         self.layernorm_3 = layers.LayerNormalization()

#         self.embedding = PositionalEmbedding(
#             embed_dim = EMBED_DIM,
#             sequence_length = SEQ_LENGTH,
#             vocab_size = VOCAB_SIZE,
#         )
#         self.out = layers.Dense(VOCAB_SIZE, activation="softmax")

#         self.dropout_1 = layers.Dropout(.3)
#         self.dropout_2 = layers.Dropout(.5)
#         self.suports_masking=True

#     def call(self, inputs, encoder_outputs, training, mask=None):
#         inputs = self.embedding(inputs)
#         causal_mask = self.get_causal_attention_mask(inputs)

#         if mask is not None:
#             padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
#             combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
#             combined_mask = tf.minimum(combined_mask, causal_mask)

#         attention_output_1 = self.attention_1(
#             query=inputs,
#             value=inputs,
#             key=inputs,
#             attention_mask=combined_mask,
#             training=training,
#         )
#         out_1 = self.layernorm_1(inputs + attention_output_1)

#         attention_output_2 = self.attention_2(
#             query=out_1,
#             value=encoder_outputs,
#             key=encoder_outputs,
#             attention_mask=padding_mask,
#             training=training,
#         )
#         out_2 = self.layernorm_2(out_1 + attention_output_2)

#         ffn_out = self.ffn_layer_1(out_2)
#         ffn_out = self.dropout_1(ffn_out, training=training)
#         ffn_out = self.ffn_layer_2(ffn_out)

#         ffn_out = self.layernorm_3(ffn_out + out_2, training=training)
#         ffn_out = self.dropout_2(ffn_out, training=training)
#         preds = self.out(ffn_out)
#         return preds

#     def get_causal_attention_mask(self, inputs):
#         input_shape = tf.shape(inputs)
#         batch_size, sequence_length = input_shape[0], input_shape[1]
#         i = tf.range(sequence_length)[:, tf.newaxis]
#         j = tf.range(sequence_length)
#         mask = tf.cast(i >= j, dtype="int32")
#         mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
#         mult = tf.concat(
#             [
#                 tf.expand_dims(batch_size, -1),
#                 tf.constant([1, 1], dtype=tf.int32),
#             ],
#             axis=0,
#         )
#         return tf.tile(mask, mult)
        

```python
def get_causal_attention_mask(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    
    # Create a lower triangular matrix of ones
    mask = tf.linalg.band_part(tf.ones((sequence_length, sequence_length)), -1, 0)
    
    # Broadcast the mask to the batch size
    mask = tf.broadcast_to(mask[None, None, :, :], (batch_size, 1, sequence_length, sequence_length))
    
    return mask
```

In [20]:
# class ImageCaptioningModel(keras.Model):
#     def __init__(
#         self, 
#         cnn_model,
#         encoder,
#         decoder,
#         num_captions_per_image=5,
#         image_aug = None
#     ):
#         super().__init__()
#         self.cnn_model = cnn_model,
#         self.encoder = encoder,
#         self.decoder = decoder,
#         self.loss_tracker = keras.metrics.Mean(name="loss")
#         self.acc_tracker = keras.metrics.Mean(name="accuracy")
#         self.num_captions_per_image = num_captions_per_image
#         self.image_aug = image_aug

#     def calculate_loss(self, y_true, y_pred, mask):
#         loss = self.loss(y_true, y_pred)
#         mask = tf.cast(mask, dtype=loss.dtype)
#         # padding loss removal
#         loss *= mask
#         # The average loss per non-padding token in the batch.
#         return tf.reduce_sum(loss) / tf.reduce_sum(mask)


#     def calculate_accuracy(self, y_true, y_pred, mask):
#         # The accuracy in image captioning is typically computed at the token level, not at the entire caption level.
#         #  (batch_size, max_sequence_length, vocabulary_size)
#         accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
#         accuracy = tf.math.logical_and(mask, accuracy)
#         acuracy = tf.cast(accuracy, dtype=tf.float32)
#         mask = tf.cast(mask, dtype=tf.float32)
#         return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

#     def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
#         encoder_out = self.encoder(img_embed, training=training)
#         batch_seq_inp = batch_seq[:, :-1]
#         batch_seq_true = batch_seq[:, 1:]
#         mask = tf.match.not_equal(batch_seq_true, 0)
#         batch_seq_pred = self.decoder(
#             batch_seq_inp,
#             encoder_out,
#             training=training,
#             mask=mask
#         )
#         loss = self.calculate_loss(batch_seq_true,
#                                    batch_seq_pred,
#                                    mask)
#         acc = self.calculate_accuracy(batch_seq_true, 
#                                       batch_seq_pred, 
#                                       mask)
#         return loss, acc


#     def train_step(self, batch_data):
#         batch_img, batch_seq = batch_data
#         batch_loss = 0
#         batch_acc = 0

#         if self.image_aug:
#             batch_img = self.image_aug(batch_img)

#         # 1. Get image embeddings
#         img_embed = self.cnn_model(batch_img)

#         # 2. Pass each of the five captions one by one to the decoder
#         # along with the encoder outputs and compute the loss as well as accuracy
#         # for each caption.
#         for i in range(self.num_captions_per_image):
#             with tf.GradientTape() as tape:
#                 loss, acc = self._compute_caption_loss_and_acc(
#                     img_embed, batch_seq[:, i, :], training=True
#                 )

#                 # 3. Update loss and accuracy
#                 batch_loss += loss
#                 batch_acc += acc

#             # 4. Get the list of all the trainable weights
#             train_vars = (
#                 self.encoder.trainable_variables + self.decoder.trainable_variables
#             )

#             # 5. Get the gradients
#             grads = tape.gradient(loss, train_vars)

#             # 6. Update the trainable weights
#             self.optimizer.apply_gradients(zip(grads, train_vars))

#         # 7. Update the trackers
#         batch_acc /= float(self.num_captions_per_image)
#         self.loss_tracker.update_state(batch_loss)
#         self.acc_tracker.update_state(batch_acc)

#         # 8. Return the loss and accuracy values
#         return {
#             "loss": self.loss_tracker.result(),
#             "acc": self.acc_tracker.result(),
#         }

#     def test_step(self, batch_data):
#         batch_img, batch_seq = batch_data
#         batch_loss = 0
#         batch_acc = 0

#         # 1. Get image embeddings
#         img_embed = self.cnn_model(batch_img)

#         # 2. Pass each of the five captions one by one to the decoder
#         # along with the encoder outputs and compute the loss as well as accuracy
#         # for each caption.
#         for i in range(self.num_captions_per_image):
#             loss, acc = self._compute_caption_loss_and_acc(
#                 img_embed, batch_seq[:, i, :], training=False
#             )

#             # 3. Update batch loss and batch accuracy
#             batch_loss += loss
#             batch_acc += acc

#         batch_acc /= float(self.num_captions_per_image)

#         # 4. Update the trackers
#         self.loss_tracker.update_state(batch_loss)
#         self.acc_tracker.update_state(batch_acc)

#         # 5. Return the loss and accuracy values
#         return {
#             "loss": self.loss_tracker.result(),
#             "acc": self.acc_tracker.result(),
#         }

#     @property
#     def metrics(self):
#         # We need to list our metrics here so the `reset_states()` can be
#         # called automatically.
#         return [self.loss_tracker, self.acc_tracker]


# cnn_model = get_cnn_model()
# encoder = TransformerEncoderBlock(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=1)
# decoder = TransformerDecoderBlock(embed_dim=EMBED_DIM, ff_dim=FF_DIM, num_heads=2)
# caption_model = ImageCaptioningModel(
#     cnn_model=cnn_model,
#     encoder=encoder,
#     decoder=decoder,
#     image_aug=image_augmentation
# )

1. **`loss = self.loss(y_true, y_pred)`:**
   - Calculates the loss between the predicted caption (y_pred) and the ground truth caption (y_true) using the model's defined loss function (self.loss). This is typically a cross-entropy loss for sequence prediction tasks.
2. **`mask = tf.cast(mask, dtype=loss.dtype)`:**
   - Ensures that the mask has the same data type as the loss for element-wise multiplication.
3. **`loss \*= mask`:**
   - Applies the mask to the loss. The mask is used to ignore padding tokens in the caption sequences. By multiplying the loss with the mask, the loss for padding tokens becomes zero, effectively excluding them from the calculation.
4. **`return tf.reduce_sum(loss) / tf.reduce_sum(mask)`:**
   - Calculates the average loss by summing the masked loss and dividing it by the total number of non-padding tokens (sum of the mask). This gives the average loss per non-padding token in the batch.

### Key Points

- The function calculates the loss for a batch of image-caption pairs.
- It incorporates a mask to ignore padding tokens and compute the loss only for actual words.
- The final output is the average loss per non-padding token in the batch.

___

```python
return tf.reduce_sum(loss) / tf.reduce_sum(mask)
```

>In essence, the mask acts as a filter, and tf.reduce_sum(mask) provides the count of elements that pass through the filter (non-padding tokens). This count is used to normalize the loss and get a meaningful average loss per non-padding token in the batch

___
### Why is `axis=2`

1. Let's consider a tensor `y_pred` with the shape `(batch_size, max_sequence_length, vocabulary_size)`.
   - **batch_size:** This dimension represents the number of image-caption pairs in the batch.
     - The first element along this dimension corresponds to the first caption and its associated data.
     - The second element corresponds to the second caption, and so on.
   - **max_sequence_length:** This dimension represents the maximum number of tokens in a caption.
   - **vocabulary_size:** This dimension represents the size of the vocabulary.
  
___
### Why `shifting` 
```python
batch_seq_inp = batch_seq[:, :-1]
batch_seq_true = batch_seq[:, 1:]
```
**Teacher Forcing** The primary reason for shifting is to implement a training technique called "teacher forcing". In teacher forcing, instead of feeding the model's previous output as input for the next prediction (like in inference), we directly feed the ground truth token. This stabilizes training and often leads to faster convergence.

**Alignment for Loss Calculation** To calculate the loss, we need to compare the predicted tokens with the corresponding ground truth tokens. By shifting `batch_seq` to create `batch_seq_true`, we ensure that the predicted tokens from the decoder align correctly with their respective ground truth tokens.

### Concrete Example

Let's consider a simplified example with a vocabulary size of 5 and a maximum sequence length of 3:

```
batch_seq = [[1, 2, 3],  # Ground truth caption: "word1 word2 word3"
            [4, 5, 0]]  # Ground truth caption: "word4 word5 <pad>"

batch_seq_inp = [[0, 1, 2],  # Input to decoder: <start> word1 word2
                [0, 4, 5]]  # Input to decoder: <start> word4 word5

batch_seq_true = [[1, 2, 3],  # Target for loss calculation: word1 word2 word3
                 [4, 5, 0]]  # Target for loss calculation: word4 word5 <pad>
```

As you can see, the decoder receives the previous token as input (along with the encoder output) and predicts the next token. The shifted `batch_seq_true` provides the correct target for calculating the loss.

**In summary, shifting `batch_seq` is essential for training sequence-to-sequence models like image captioning. It enables teacher forcing and ensures proper alignment for loss calculation.**


# important note
### Teacher Forcing

- **Training phase:** Used to stabilize training and accelerate convergence.
- **Process:** The ground truth token is fed as input to the decoder at each time step.

### Autoregression

- **Inference phase:** Used to generate text sequentially, one token at a time.
- **Process:** The model's own previous output is fed as input to predict the next token.

**In essence, teacher forcing is a training technique, while autoregression is the fundamental mechanism for generating text sequentially.**

### Combining Both

It's common to use teacher forcing during training and then switch to autoregressive mode during inference. However, there are also techniques like scheduled sampling that gradually transition from teacher forcing to autoregression during training to improve model robustness.

In [21]:
# # Define the loss function
# cross_entropy = keras.losses.SparseCategoricalCrossentropy(
#     from_logits=False,
#     reduction=None,
# )

# # EarlyStopping criteria
# early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)


# # Learning Rate Scheduler for the optimizer
# class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
#     def __init__(self, post_warmup_learning_rate, warmup_steps):
#         super().__init__()
#         self.post_warmup_learning_rate = post_warmup_learning_rate
#         self.warmup_steps = warmup_steps

#     def __call__(self, step):
#         global_step = tf.cast(step, tf.float32)
#         warmup_steps = tf.cast(self.warmup_steps, tf.float32)
#         warmup_progress = global_step / warmup_steps
#         warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
#         return tf.cond(
#             global_step < warmup_steps,
#             lambda: warmup_learning_rate,
#             lambda: self.post_warmup_learning_rate,
#         )


# # Create a learning rate schedule
# num_train_steps = len(train_dataset) * EPOCHS
# num_warmup_steps = num_train_steps // 15
# lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# # Compile the model
# caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)

# # Fit the model
# caption_model.fit(
#     train_dataset,
#     epochs=EPOCHS,
#     validation_data=valid_dataset,
#     callbacks=[early_stopping],
# )

In [53]:
def get_cnn_model():
    base_model = efficientnet.EfficientNetB0(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        weights="imagenet",
    )
    # We freeze our feature extractor
    base_model.trainable = False
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model


class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.0
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.dense_1 = layers.Dense(embed_dim, activation="relu")

    def call(self, inputs, training, mask=None):
        inputs = self.layernorm_1(inputs)
        inputs = self.dense_1(inputs)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            training=training,
        )
        out_1 = self.layernorm_2(inputs + attention_output_1)
        return out_1


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.embed_scale = tf.math.sqrt(tf.cast(embed_dim, tf.float32))

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_tokens = embedded_tokens * self.embed_scale
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_layer_2 = layers.Dense(embed_dim)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.embedding = PositionalEmbedding(
            embed_dim=EMBED_DIM,
            sequence_length=SEQ_LENGTH,
            vocab_size=VOCAB_SIZE,
        )
        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")

        self.dropout_1 = layers.Dropout(0.3)
        self.dropout_2 = layers.Dropout(0.5)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=combined_mask,
            training=training,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            training=training,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.dropout_1(ffn_out, training=training)
        ffn_out = self.ffn_layer_2(ffn_out)

        ffn_out = self.layernorm_3(ffn_out + out_2, training=training)
        ffn_out = self.dropout_2(ffn_out, training=training)
        preds = self.out(ffn_out)
        return preds

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32),
            ],
            axis=0,
        )
        return tf.tile(mask, mult)


class ImageCaptioningModel(keras.Model):
    def __init__(
        self,
        cnn_model,
        encoder,
        decoder,
        num_captions_per_image=5,
        image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        mask = tf.math.not_equal(batch_seq_true, 0)
        batch_seq_pred = self.decoder(
            batch_seq_inp, encoder_out, training=training, mask=mask
        )
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        # 1. Get image embeddings
        print(batch_img.shape)
        batch_img = np.expand_dims(batch_img, axis=0)
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )

                # 3. Update loss and accuracy
                batch_loss += loss
                batch_acc += acc

            # 4. Get the list of all the trainable weights
            train_vars = (
                self.encoder.trainable_variables + self.decoder.trainable_variables
            )

            # 5. Get the gradients
            grads = tape.gradient(loss, train_vars)

            # 6. Update the trainable weights
            self.optimizer.apply_gradients(zip(grads, train_vars))

        # 7. Update the trackers
        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 8. Return the loss and accuracy values
        return {
            "loss": self.loss_tracker.result(),
            "acc": self.acc_tracker.result(),
        }

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {
            "loss": self.loss_tracker.result(),
            "acc": self.acc_tracker.result(),
        }

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker, self.acc_tracker]


cnn_model = get_cnn_model()
encoder = TransformerEncoderBlock(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=1)
decoder = TransformerDecoderBlock(embed_dim=EMBED_DIM, ff_dim=FF_DIM, num_heads=2)
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model,
    encoder=encoder,
    decoder=decoder,
    image_aug=image_augmentation,
)
# print(caption_model.summary())

In [54]:
cnn_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 229, 229,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_12        │ (None, 229, 229,  │          0 │ input_layer_7[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_6     │ (None, 229, 229,  │          7 │ rescaling_12[0][… │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_13        │ (None, 229, 229,  │          0 │ normalization_6[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 231, 231,  │          0 │ rescaling_13[0][… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 115, 115,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 115, 115,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 115, 115,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 115, 115,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 115, 115,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 115, 115,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 115, 115,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 115, 115,  │        512 │ block1a_se_excit

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [55]:
# Define the loss function
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    reduction=None,
)

# EarlyStopping criteria
early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)


# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,
        )


# Create a learning rate schedule
num_train_steps = len(train_dataset) * EPOCHS
num_warmup_steps = num_train_steps // 15
lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# Compile the model
caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)

# Fit the model
caption_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=[early_stopping],
)

Epoch 1/30
(229, 229, 3)


NotImplementedError: Cannot convert a symbolic tf.Tensor (sequential_1/random_contrast_1/clip_by_value:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.